In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/Users/quansun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [35]:
# Parameters

INPUT_NODE = 784 # 28*28
OUTPUT_NODE = 10 # ten numbers from 0 to 9
LAYER1_NODE = 500 # The number of neurons in 1st hiddden layer
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001 # The coefficient of regularization term
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99

In [36]:
def inference(input_tensor, avg_class, weights1,biases1,
              weights2,biases2):
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor,weights1)+biases1)
        return tf.matmul(layer1,weights2)+biases2
    else:
        layer1=tf.nn.relu(
            tf.matmul(input_tensor,avg_class.average(weights1))+
            avg_class.average(biases1))
        return tf.matmul(layer1,avg_class.average(weights2)) + \
                avg_class.average(biases2)

In [37]:
def train(mnist):
    x = tf.placeholder(tf.float32,[None,INPUT_NODE],name='x_input')
    y_ = tf.placeholder(tf.float32,[None,10],name='y_input')
    
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE,LAYER1_NODE],stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1,shape=[LAYER1_NODE]))
    
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE,OUTPUT_NODE],stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1,shape=[OUTPUT_NODE]))
    
    # We can also use a better initializer for weights- that is xavier
    # initializer = tf.contrib.layers.variance_scaling_initializer(
    # factor=2.0, mode='FAN_IN', uniform=False)
    
    y = inference(x,None,weights1,biases1,weights2,biases2)
    
    global_step = tf.Variable(0,trainable=False) # define the step of training
    
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    
    # Apply moving average to all parameters except global_step
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    
    average_y = inference(x,variable_averages,weights1,biases1,weights2,biases2)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # Loss function with L2 regularization
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularization = regularizer(weights1)+regularizer(weights2)
    loss = cross_entropy_mean + regularization
    
    # Set learning_rate
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step, # current step
        mnist.train.num_examples/BATCH_SIZE, # the number of iteration after itering all data
        LEARNING_RATE_DECAY)
    
    train_step = tf.train.GradientDescentOptimizer(learning_rate)\
                .minimize(loss,global_step=global_step)
    
    # We need update parameters as well as moving_average every iteration
    # 2 operation are useful. tf.control_dependencies, tf.group
    # train_op = tf.group(train_step,variables_averages_op)
    with tf.control_dependencies([train_step,variables_averages_op]):
        train_op = tf.no_op(name='train')
        
    correct_prediction = tf.equal(tf.argmax(average_y,1),tf.argmax(y_,1))
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    # Initialization and training
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        
        validation_feed = {x:mnist.validation.images, y_:mnist.validation.labels}
        
        test_feed = {x:mnist.test.images, y_:mnist.test.labels}
        
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:
                validation_acc = sess.run(accuracy,feed_dict=validation_feed)
                print("After %d training steps, validation accuracy using average model is %f" % (i, validation_acc))
                
            batch_x,batch_y = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op,feed_dict={x:batch_x,y_:batch_y})
            
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print("After %d training steps, test accuracy using average model is %f" % (TRAINING_STEPS, test_acc))

In [38]:
def main(argv=None):
    mnist = input_data.read_data_sets('./mnist',one_hot=True)
    train(mnist)

In [39]:
if __name__ == '__mian__':
    tf.app.run()

In [ ]:
main()

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
After 0 training steps, validation accuracy using average model is 0.166000
After 1000 training steps, validation accuracy using average model is 0.976800
